In [ ]:
!pip install -qq pdfplumber openpyxl regex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 52.3 MB/s eta 0:00:00


In [ ]:
import pdfplumber
import regex
import pandas as pd
from google.colab import files

In [ ]:
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]  # nome do arquivo PDF
print("PDF carregado:", pdf_path)

Saving CONCARGA.pdf to CONCARGA.pdf
PDF carregado: CONCARGA.pdf


In [ ]:
extracted_data = []

In [ ]:
servidor_regex = (
    r"^([\p{L}'\s\.]+)"   # Nome do servidor (caracteres UNICODE, espaços, etc.)
    r"\s+"
    r"(MAT|VES)"          # Turno
    r"\s+(\d{2})"         # CH do servidor
    r"\s+(\d{11,})$"      # Código SI (11+ dígitos)
)

In [ ]:
# Regex para segunda linha do servidor, ex.:
# "Matric.: 01127629-8G   VINCULO: A  Hab.: INTE ARTES"
mvh_regex = (
    r"Matric\.\:\s+([\d\-A-Z]+)\s+"   # Matrícula
    r"VINCULO\:\s+([A-Z])\s+"        # Vínculo
    r"Hab\.\:\s+(.+)$"               # Habilitação (resto da linha)
)

In [ ]:
# Regex para disciplina, mas agora em findall
# Exemplo de pedaço: "039 FUND 09 05 0893 ART 1"
# Precisamos capturar 6 grupos:
#   1) 039 FUND  -> Ex.: "039 FUND"
#   2) 09        -> Série/Ano
#   3) 05        -> Turma
#   4) 0893      -> Cód. Disc
#   5) ART       -> Disc (p. ex. "ART")
#   6) 1         -> CH.TURMA
#
# Se a disciplina tiver 2 palavras ("CIEN NATUR"), será preciso ajustar
disc_pattern = (
    r"(0\d{2}\s[\p{L}]+)\s+(\d{2})\s+(\d{2})\s+(\d{4,})\s+([\p{L}]+)\s+(\d+)"
)

In [ ]:
# =============================================================
# 5) LER O PDF, PÁGINA A PÁGINA
# =============================================================
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if not text:
            continue

        lines = text.split("\n")
        i = 0

        # Percorre as linhas
        while i < len(lines):
            line = lines[i].strip()

            # -----------------------------------------------------
            # VERIFICA SE É LINHA DE SERVIDOR
            # -----------------------------------------------------
            match_servidor = regex.match(servidor_regex, line, flags=regex.IGNORECASE)
            if match_servidor:
                # Captura de dados do servidor
                nome_servidor = match_servidor.group(1).strip()
                turno = match_servidor.group(2).strip()
                ch_servidor = match_servidor.group(3).strip()
                codigo_si = match_servidor.group(4).strip()

                # Captura matrícula, vínculo, habilitação na linha seguinte
                matricula = ""
                vinculo = ""
                habilitacao = ""

                if (i + 1) < len(lines):
                    line_next = lines[i + 1].strip()
                    mvh_match = regex.search(mvh_regex, line_next)
                    if mvh_match:
                        matricula = mvh_match.group(1).strip()
                        vinculo = mvh_match.group(2).strip()
                        habilitacao = mvh_match.group(3).strip()

                # Percorre as linhas posteriores para achar disciplinas
                j = i + 2
                while j < len(lines):
                    line_disc = lines[j].strip()

                    # Se linha de separador ou outro servidor, paramos este bloco
                    if line_disc.startswith("---"):
                        break
                    if regex.match(servidor_regex, line_disc, flags=regex.IGNORECASE):
                        break

                    # Usa findall para capturar TODAS as disciplinas nesta linha
                    disc_matches = regex.findall(disc_pattern, line_disc)

                    # Se acharmos 1 ou mais disciplinas na linha, salvamos cada uma
                    if disc_matches:
                        for dm in disc_matches:
                            # dm é uma tupla com 6 elementos
                            # Ex.: ("039 FUND","09","05","0893","ART","1")
                            ensino      = dm[0]
                            serie_ano   = dm[1]
                            turma       = dm[2]
                            cod_disc    = dm[3]
                            nome_disc   = dm[4]
                            ch_turma    = dm[5]

                            extracted_data.append({
                                "SERVIDOR": nome_servidor,
                                "TURNO": turno,
                                "CH": ch_servidor,
                                "MATRICULA": matricula,
                                "VINCULO": vinculo,
                                "HABILITACAO": habilitacao,
                                "ENSINO": ensino,
                                "SERIE/ANO": serie_ano,
                                "TURMA": turma,
                                "COD.DISC": cod_disc,
                                "DISC": nome_disc,
                                "CH.TURMA": ch_turma
                            })

                    j += 1

                # Avança o i para pular essas linhas já analisadas
                i = j
            else:
                i += 1

In [ ]:
# =============================================================
# 6) CONVERTER LISTA DE DICIONÁRIOS EM DATAFRAME
# =============================================================
df = pd.DataFrame(extracted_data)

In [ ]:
# =============================================================
# 7) SEPARAR "ENSINO" EM "COD.ESCOLA" e "NIVEL"
# Se 'ENSINO' = '039 FUND', split em duas partes.
# ATENÇÃO: se houver 3 palavras (ex. "039 FUND AB"), será preciso adaptar.
# =============================================================
# Tenta split com n=1: '039 FUND' -> ('039', 'FUND')
df[["COD.ESCOLA", "NIVEL"]] = df["ENSINO"].str.split(n=1, expand=True)
df.drop(columns=["ENSINO"], inplace=True)

In [ ]:
# =============================================================
# 8) REORDENAR COLUNAS (se desejar)
# =============================================================
colunas_finais = [
    "SERVIDOR",
    "TURNO",
    "CH",
    "MATRICULA",
    "VINCULO",
    "HABILITACAO",
    "COD.ESCOLA",
    "NIVEL",
    "SERIE/ANO",
    "TURMA",
    "COD.DISC",
    "DISC",
    "CH.TURMA"
]
df = df[colunas_finais]

In [ ]:
# Converter as colunas 'CH' e 'CH.TURMA' para inteiros
df["CH"] = pd.to_numeric(df["CH"], errors="coerce").astype("Int64")
df["CH.TURMA"] = pd.to_numeric(df["CH.TURMA"], errors="coerce").astype("Int64")

In [ ]:
df.head(20)

                  SERVIDOR TURNO  CH    MATRICULA VINCULO HABILITACAO  \
0   THASSIA CRISTINA DA SI   MAT  20  01271340-3A       F  PEDAGOG LI   
1       DIANA RIBEIRO TOME   MAT  15  01214971-0B       A  LETRASPORF   
2       DIANA RIBEIRO TOME   MAT  15  01214971-0B       A  LETRASPORF   
3       DIANA RIBEIRO TOME   MAT  15  01214971-0B       A  LETRASPORF   
4   ELENICE MARIA DE OLIVE   MAT  15  01183903-9B       A  LETRASPORF   
5   ELENICE MARIA DE OLIVE   MAT  15  01183903-9B       A  LETRASPORF   
6   ELENICE MARIA DE OLIVE   MAT  15  01183903-9B       A  LETRASPORF   
7   GLAURIA GLEICE GAMA DO   MAT  15  01217267-4A       A  LETRASPORF   
8   GLAURIA GLEICE GAMA DO   MAT  15  01217267-4A       A  LETRASPORF   
9   GLAURIA GLEICE GAMA DO   MAT  15  01217267-4A       A  LETRASPORF   
10  ANCELMO DE AZEVEDO DOS   MAT  15  01224299-0C       A  LETRASPORF   
11  ANCELMO DE AZEVEDO DOS   MAT  15  01224299-0C       A  LETRASPORF   
12  ANCELMO DE AZEVEDO DOS   MAT  15  01224299-0C  

In [ ]:
saida_excel = "saida_final.xlsx"
df.to_excel(saida_excel, index=False)
print(f"Planilha gerada: {saida_excel}")

Planilha gerada: saida_final.xlsx


In [ ]:
# Selecionar as colunas necessárias
df_filtered = df[['SERVIDOR', 'TURNO']].drop_duplicates()

In [ ]:
# Criar subdivisões das colunas SEGUNDA a SEXTA (de 1 a 5) e preenchê-las com 0
dias_semana = ['SEG', 'TER', 'QUA', 'QUI', 'SEX']
for dia in dias_semana:
    for i in range(1, 6):  # Criando colunas SEG1, SEG2, ..., SEX5
        df_filtered[f"{dia}{i}"] = 0

In [ ]:
df_filtered.head(20)

,SERVIDOR,TURNO,SEG1,SEG2,SEG3,SEG4,SEG5,TER1,TER2,TER3,...,QUI1,QUI2,QUI3,QUI4,QUI5,SEX1,SEX2,SEX3,SEX4,SEX5
0,THASSIA CRISTINA DA SI,MAT,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,DIANA RIBEIRO TOME,MAT,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ELENICE MARIA DE OLIVE,MAT,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,GLAURIA GLEICE GAMA DO,MAT,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,ANCELMO DE AZEVEDO DOS,MAT,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,ANDREA CRISTINA NASCIM,MAT,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16,MIRAELZA MACEDO DE ASS,MAT,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,SUELY DOS SANTOS ALMEI,MAT,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,LAERCIO RODRIGO OLIVEI,MAT,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,LIANA FERREIRA MAGALHA,MAT,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Caminho para salvar a nova planilha
output_file = "/content/professores.xlsx"

# Salvar a planilha sem valores duplicados
df_filtered.to_excel(output_file, index=False)